**과정**

    1. 초기화

In [326]:
import pandas as pd
import random

In [511]:
#초깃값
size = 10 #10 대신 list 크기
inits = size #유전자 갯수
repeat = 3 #반복 횟수

lists = [[0]*size]*inits

pd_list = pd.DataFrame(lists)
pd_list = pd_list.applymap(lambda x : random.randint(0,1))

    2. 선택

In [512]:
#Rating 함수, 외부 입력 여기선 1의 갯수로
def rat(x):
    res = 0
    for i in x:
        res += i
    return res

In [524]:
rating = rat
rates = {}
for item in range(inits):
    rates[item] = rating(pd_list.irow(item))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
  after removing the cwd from sys.path.


In [525]:
import operator 
sorteds = sorted(rates.items(), key=operator.itemgetter(1), reverse=True) 
selects = [None]*int(inits/2)
for i in range(int(inits/2)):
    selects[i] = pd_list.irow(sorteds[i][0])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
  """


In [515]:
selects

[0    1
 1    1
 2    0
 3    1
 4    1
 5    1
 6    1
 7    1
 8    0
 9    1
 Name: 4, dtype: int64, 0    1
 1    0
 2    1
 3    1
 4    1
 5    0
 6    0
 7    1
 8    1
 9    1
 Name: 1, dtype: int64, 0    0
 1    0
 2    0
 3    1
 4    1
 5    1
 6    1
 7    1
 8    1
 9    1
 Name: 5, dtype: int64, 0    1
 1    1
 2    1
 3    1
 4    0
 5    1
 6    0
 7    1
 8    1
 9    0
 Name: 8, dtype: int64, 0    1
 1    0
 2    0
 3    0
 4    1
 5    0
 6    1
 7    1
 8    1
 9    1
 Name: 0, dtype: int64]

    3.교차

In [516]:
import numpy

In [517]:
crossed = [[0]*size]*inits

crossed_list = pd.DataFrame(crossed)
crossed_list = crossed_list.applymap(lambda x : random.randint(0,len(selects)-1))

In [518]:
def selector(x):
    res = []
    for i in x:
        res.append(selects[i][int(x.name)])
    return pd.core.series.Series(data = res)

In [519]:
crossed_list = crossed_list.apply(selector)# + x.name*10)#selects[int(x[0])][int(x.name)])#x + x.name*10)#selects[x][int(x)])0

    5.변이

In [520]:
for i in range(0, int(inits/2)):
    set_x = random.randint(0,inits -1)
    set_y = random.randint(0,size -1)
    crossed_list[set_y][set_x] ^= 1

In [521]:
crossed_list

,0,1,2,3,4,5,6,7,8,9
0,1,0,0,1,1,1,1,1,1,1
1,1,0,0,0,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,0,1,1,1
4,1,1,0,0,1,0,0,1,1,1
5,1,1,0,1,1,1,1,1,1,1
6,0,1,1,1,1,1,1,1,1,1
7,0,0,1,1,1,0,1,1,1,1
8,1,0,1,1,1,0,1,1,1,1
9,1,1,0,0,1,0,0,1,1,1


    6.대치

In [522]:
lists = crossed_list

In [523]:
lists

,0,1,2,3,4,5,6,7,8,9
0,1,0,0,1,1,1,1,1,1,1
1,1,0,0,0,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,0,1,1,1
4,1,1,0,0,1,0,0,1,1,1
5,1,1,0,1,1,1,1,1,1,1
6,0,1,1,1,1,1,1,1,1,1
7,0,0,1,1,1,0,1,1,1,1
8,1,0,1,1,1,0,1,1,1,1
9,1,1,0,0,1,0,0,1,1,1


In [1]:

# coding: utf-8

# **과정**

#     1. 초기화

# In[326]:
from multiprocessing import Pool
import pandas as pd
import random
import operator 
import numpy

class genetic:
    # In[511]:
    #	size# = 10 #10 대신 list 크기
    #inits# = size #유전자 갯수
    #repeat# = 3 #반복 횟수

    #Rating 함수, 외부 입력 여기선 1의 갯수로
    def rat(self, x):
        res = 0
        for i in x:
            res += i
        return res


    def __init__(self,size, inits, repeat, rat_func, n_jobs=1):
        #초깃값
        self.size = size
        self.inits = inits
        self.repeat = repeat
        self.rat_func = rat_func
        self.lists = [[0]*size]*inits
        
        self.pd_list = pd.DataFrame(self.lists)
        self.pd_list = self.pd_list.applymap(lambda x : random.randint(0,1))

        self.pool =  Pool(processes=n_jobs)
        
    def run(self):
        for i in range(self.repeat):
            self.do()

    #def rating(self, item):
    #    return (item, self.rat_func(self.pd_list.irow(item)))
    
    def generate_row(self):
        for i in range(self.inits):
            yield self.pd_list.irow(i)
    
    def do(self):
        #self.rating = self.rat_func
        self.rates = {}
        
        i = 0
        for result in self.pool.map(self.rat_func, self.generate_row()):
            self.rates[i] = result
            i+=1
        
        
        # In[514]:

        self.sorteds = sorted(self.rates.items(), key=operator.itemgetter(1), reverse=True) 
        self.selects = [None]*int(self.inits/2)
        for i in range(int(self.inits/2)):
            self.selects[i] = self.pd_list.irow(self.sorteds[i][0])


    #     3.교차
        self.crossed = [[0]*self.size]*self.inits

        self.crossed_list = pd.DataFrame(self.crossed)
        self.crossed_list = self.crossed_list.applymap(lambda x : random.randint(0,len(self.selects)-1))


    # In[518]:

        def selector(x):
            res = []
            for i in x:
                res.append(self.selects[i][int(x.name)])
            return pd.core.series.Series(data = res)


    # In[519]:

        self.crossed_list = self.crossed_list.apply(selector)# + x.name*10)#selects[int(x[0])][int(x.name)])#x + x.name*10)#selects[x][int(x)])0


    #     5.변이

    # In[520]:

        for i in range(0, int(self.inits/2)):
            set_x = random.randint(0,self.inits -1)
            set_y = random.randint(0,self.size -1)
            self.crossed_list[set_y][set_x] ^= 1

        self.pd_list = self.crossed_list


    def done(self):
        #for item in range(self.inits):
        #	self.rates[item] = self.rating(self.pd_list.irow(item))
        # In[514]:
        #self.sorteds = sorted(self.rates.items(), key=operator.itemgetter(1), reverse=True) 
        #selects = [None]*int(self.inits/2)
        for i in range(int(self.inits/2)):
            #self.selects[i] = self.pd_list.irow(self.sorteds[i][0])
            print(self.selects[i])
        return self.selects

In [3]:
def rat(x):
    res = 0
    for i in x:
        res += i
    return -1*abs(5-res)

a = genetic(100,30,20, rat, n_jobs=26)

In [4]:
a.run()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:68: FutureWarning: irow(i) is deprecated. Please use .iloc[i]


In [5]:
rat(a.done()[0])

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
     ..
70    0
71    0
72    0
73    0
74    0
75    1
76    0
77    0
78    0
79    0
80    0
81    0
82    0
83    0
84    1
85    0
86    0
87    1
88    0
89    0
90    0
91    0
92    0
93    0
94    0
95    0
96    0
97    0
98    0
99    0
Name: 11, dtype: int64
0     0
1     0
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
     ..
70    0
71    0
72    0
73    0
74    0
75    0
76    0
77    0
78    0
79    0
80    0
81    0
82    1
83    0
84    0
85    0
86    0
87    1
88    0
89    0
90    0
91    0
92    0
93    0
94    0
95    0
96    0
97    0
98    0
99    0
N

0